In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv('polarity_labeling.csv')
data.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,asin,date,product,review,score,user,CS_SS,target,feature,polarity
0,B00000JFOA,"01 6, 2000",c-2000,the newer model c-2020 has all the fixes exce...,4.0,Lee,CS,c-2020,lens,0
1,B000023VQC,"06 24, 2001",pdc-700,it likes to change from xga to vga etc.. *the ...,2.0,A. Gray,CS,vga,battery,0
2,B00003OPEY,"04 26, 2000",dsc-f505,the included memory stick is a little small b...,5.0,David Pirmann,CS,110,memory,1
3,B00004R8V6,"09 5, 2000",d-360l,it also adds a nifty little optical function ...,5.0,"Baltic Books ""Vic""",CS,d-460,"optical,battery,lcd,compression,flash",1
4,B00004R8VC,"04 18, 2000",d-460,i read a lot of reviews on the d450 optical be...,5.0,JR,CS,d450,optical,-1


In [ ]:
df = data[data['polarity'] != 0]
len(df)

In [4]:
def Make_matrix(df):
    product_list = data['product'].unique()
    matrix = np.zeros((len(product_list), len(product_list)), dtype=np.float32)
    
    df = df[df['polarity'] != 0]
    for i, p in enumerate(product_list):
        df_re = df[df['product'] == p]
        df_cs = df_re[df_re['CS_SS'] == 'cs']
        df_ss = df_re[df_re['CS_SS'] == 'ss']
        
        if i%10 == 0:
            print(i)
        
        for j, t in enumerate(product_list):
            # ss에 의한 웨이트 계산
            ss_data = df_ss['polarity'].value_counts()
            if len(ss_data.index) == 2:
                matrix[i,i] = int(ss_data[ss_data.index == 1]) / int(ss_data[ss_data.index == -1])
            elif 1 not in ss_data.index:
                matrix[i,i] = 0
            else:
                matrix[i,i] = int(ss_data[ss_data.index == 1])

        else:
            # cs에 의한 웨이트 계산
            cs_data = df_cs[df_cs['target'].str.contains(t)]['polarity'].value_counts()
            if len(cs_data.index) == 2:
                matrix[i,j] = int(cs_data[cs_data.index == 1]) / int(cs_data[cs_data.index == -1])
            elif 1 not in cs_data.index:
                matrix[i,j] = 0
            else:
                matrix[i,j] = int(cs_data[cs_data.index == 1])
    
    dff = pd.DataFrame(matrix, index = product_list, columns = product_list)
    dff.loc['sum'] = dff.sum()
    dff.tail()
    
    return dff

In [5]:
df = Make_matrix(data)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220


In [ ]:
dff = pd.DataFrame(a, index = product_list, columns = product_list)
dff.loc['sum'] = dff.sum()

In [ ]:
dff.tail()

In [ ]:
score = np.ones(len(product_list))
score

In [ ]:
def Make_Rank(df, d = 0.85, MAX_ITERATIONS = 50, threshold = 0.0001):
    score = np.ones(len(product_list))

    for iter in range(0,MAX_ITERATIONS):
        prev_score = np.copy(score)
        
        for i, p in enumerate(product_list):
            summation = 0
            for j, t in enumerate(product_list):
                if df.loc['sum', t] != 0:
                    summation += df.loc[p, t] / df.loc['sum', t] * score[j]

            score[i] = (1-d) + d*summation

        if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
            print('요기서 끝 :', iter)
            break
    
    dff = pd.DataFrame(score, index = product_list)
    sum_num = float(dff.sum())
    dff = dff.apply(lambda x: x/sum_num)
    
    return dff

In [ ]:
a = Make_Rank(dff)
a